In [ ]:
import cmipld
from pyld import jsonld
import os, re, json, glob

In [ ]:
for i in cmipld.mapping:
    
    try:
        jsonld.expand(f'{cmipld.mapping[i]}graph.jsonld')
    except Exception as e:
        print(i.capitalize())
        print(e)
        print(f'{cmipld.mapping[i]}graph.jsonld')
        print('---')

In [ ]:
# import requests
# from pprint import pprint

# def fetch_repo_tree(owner, repo, path="", base=None, name="root", level = 0):
#     printpath = ""
#     hierarchy = {}
#     url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
#     base = base or f"{path}/"
    
#     response = requests.get(url)
#     if response.status_code == 200:
#         contents = response.json()
        
#         hierarchy[name] = dict(name = name, path = base, children = [])
        
#         print(contents)
        
#         for item in contents:
            
            
#             printpath += f"{'|   ' * path.count('/')}├── {item['name']}\n"
#             newbase = base + item['path']
            
#             if item['type'] == "dir":
#                 pp, child = fetch_repo_tree(owner, repo, item['path'], newbase, item['name'], level+1  )
#                 printpath+=pp
#                 hierarchy[name]['children'].append(child)
                
                    
#     else:
#         print(f"Failed to fetch {url}: {response.status_code} {response.reason}")
        
#     if level == 0:  
#         print(printpath)
        
#     return printpath, hierarchy

# # Replace with your target repository owner and name
# fetch_repo_tree("WCRP-CMIP", "WCRP-UNIVERSE","src-data")

In [144]:
def get_repo_tree(owner, repo, path="src-data", branch = "main"):
    # owner,repo,path,branch = "WCRP-CMIP", "WCRP-UNIVERSE","src-data","main"
    
    if path[-1] == "/": path = path[:-1]
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}/?ref={branch}"


    def tree(url):
        
        owner,repo = url.replace('https://github.com/','').split('/',2)[:2]
        preurl = f"https://{owner}.github.io/{repo}/"
        
        response = requests.get(url)
        contents = response
        if response.status_code == 200:
            contents = response.json()
            
        owner,repo = url.replace('https://api.github.com/repos/','').split('/',2)[:2]
        preurl = f"https://{owner}.github.io/{repo}/"

        for item in contents:
            if '_links' in item: del item['_links']
            item['path']= item['path'].replace(path, '')
            item['jsonld'] = f"{preurl}{item['path']}".split('.json')[0]
            if item['type'] == "dir":
                item['children'] = tree(item['url'])
                break
                
        return contents

    complete = {
        "name":repo,
        "path":path,
        "html_url":f"https://github.com/{owner}/{repo}/blob/{branch}/{path}",
        "git_url":url,
        "children":contents
    }

    return complete

In [145]:
# contents = tree(url)

In [ ]:
# complete = {
#     "name":repo,
#     "path":path,
#     "html_url":f"https://github.com/{owner}/{repo}/blob/{branch}/{path}",
#     "git_url":url,
#     "children":contents
# }

# pprint(complete)

In [ ]:

def drawtree(hierarchy):
    printpath = ""
    for item in hierarchy:
        printpath += f"{'|   ' * item['path'].count('/')}- {item['name']}\n"
        if 'children' in item:
            printpath += drawtree(item['children'])
                

    return printpath

pprint(drawtree(contents))

In [ ]:
owner,repo

In [ ]:
def drawtree(hierarchy, level=0, start_level=0):
    """
    Generate a tree diagram with dashes (like dust).

    Args:
        hierarchy (list): A list of dictionaries representing the hierarchical structure.
        level (int): The current depth of recursion (used for indentation).
        start_level (int): The base level to start the tree (affects initial indentation).
    
    Returns:
        str: A string representing the tree diagram.
    """
    tree = ""
    for item in hierarchy:
        # Adjust indentation based on level and starting level
        indentation = "├" + "───" * (level - start_level)
        tree += f"{indentation}─── {item['name']}\n"
        if 'children' in item:
            # Recursively process children with an increased level
            tree += drawtree(item['children'], level + 1, start_level)
    return tree

pprint(drawtree(contents,0))

In [ ]:

def showall(contents):
    # Display hierarchy with different styles
    print("Tree Style:\n")
    print(display_hierarchy(contents, style="tree"))

    print("\nArrow Style:\n")
    print(display_hierarchy(contents, style="arrow"))

    print("\nIndented List Style:\n")
    print(display_hierarchy(contents, style="indented"))

    print("\nBracketed Style:\n")
    print(display_hierarchy(contents, style="bracketed"))

    print("\nTable Style:\n")
    print(display_hierarchy(contents, style="table"))

    print("\nNumbered Outline Style:\n")
    print(display_hierarchy(contents, style="numbered"))

    print("\nGraph Style:\n")
    print(display_hierarchy(contents, style="graph"))

    print("\nBullet Point Style:\n")
    print(display_hierarchy(contents, style="bullets"))

    print("\nJSON-Like Style:\n")
    print(display_hierarchy(contents, style="json"))


In [ ]:
def display_hierarchy(hierarchy, style="arrow", level=0, prefix="", start_level=0,show = False):
    """
    Display a hierarchy in various styles.

    Args:
        hierarchy (list): Hierarchical data (list of dicts with 'name' and optionally 'children').
        style (str): The style of display. Options are:
            - "tree" (default): Tree-like with branches.
            - "indented": Simple indented list.
            - "bracketed": Square bracket notation.
            - "table": ASCII table style.
            - "numbered": Numbered outline format.
            - "graph": Arrows to show relationships.
            - "bullets": Bullet points for levels.
            - "arrow": Downward-right arrow for nesting.
            - "json": JSON-like structure.
        level (int): Current recursion level for indentation (internal use).
        prefix (str): Prefix used for numbering or custom markers (internal use).
        start_level (int): Base level for indentation.

    Returns:
        str: The formatted hierarchy as a string.
    """
    output = ""

    for i, item in enumerate(hierarchy, 1):
        # Determine prefixes and indentation
        current_prefix = f"{prefix}{i}" if style == "numbered" else ""
        indent = "    " * (level - start_level)
        marker = ""
        
        if style == "tree":
            marker = f"{'│   ' * (level - start_level)}├── {item['name']}"
        elif style == "indented":
            marker = f"{'  ' * (level - start_level)}- {item['name']}"
        elif style == "bracketed":
            marker = f"{'  ' * (level - start_level)}[{item['name']}]"
        elif style == "table":
            marker = f"{indent}{item['name']}"
        elif style == "numbered":
            marker = f"{current_prefix} {item['name']}"
        elif style == "graph":
            marker = f"{'  ' * (level - start_level)}-> {item['name']}"
        elif style == "bullets":
            marker = f"{'*' * (level - start_level + 1)} {item['name']}"
        elif style == "arrow":
            marker = f"{'    ' * (level - start_level)}↳ {item['name']}"
        elif style == "json":
            marker = f"{indent}{{'name': '{item['name']}'"
            if 'children' in item:
                marker += f", 'children': [\n{display_hierarchy(item['children'], style, level + 1, prefix, start_level)}\n{indent}]"
            marker += f"}}"
        
        # Append the marker to output
        if style != "json":
            output += f"{marker}\n"
        else:
            output += f"{marker}\n"

        # Handle children recursively
        if 'children' in item and style != "json":
            output += display_hierarchy(item['children'], style, level + 1, current_prefix + ".", start_level)

    if show:
    #     print(output)
        print(output)
    return output






In [ ]:
display_hierarchy(contents, style="arrow", show = True)

In [ ]:
# contents[3]

In [ ]:
def filter_nested_dict(data, valid_extensions=None, valid_names=None):
    """
    Recursively filters a nested dictionary to include only items whose names 
    have valid extensions or match specific names.
    
    Args:
        data (list): A list of dictionaries representing the nested structure.
        valid_extensions (list): List of valid file extensions (e.g., ['.json', '.jsonld']).
        valid_names (list): List of specific names to include (e.g., ['_context_']).
    
    Returns:
        list: Filtered nested dictionary.
    """
    if valid_extensions is None:
        valid_extensions = []
    if valid_names is None:
        valid_names = []

    def is_valid(name):
        # Check if the name is valid based on extensions or specific names
        return (
            any(name.endswith(ext) for ext in valid_extensions)
            or name in valid_names
        )

    filtered = []
    for item in data:
        if is_valid(item['name']) or item['type'] != "file":
            # Keep valid items and filter their children recursively
            if 'children' in item:
                item['children'] = filter_nested_dict(
                    item['children'], valid_extensions, valid_names
                )
            filtered.append(item)
    return filtered


# Filter to keep only .json, .jsonld, and '_context_'
filtered_contents = filter_nested_dict(
    contents,
    valid_extensions=[".json", ".jsonld"],
    valid_names=["_context_"]
)


display_hierarchy(filtered_contents,show = True)


In [ ]:
# contents

In [ ]:
import json 
json.dump(contents,open('test.json','w'))

In [ ]:

def display_hierarchy(hierarchy, style="arrow", level=0, prefix="", start_level=0,show = False):
    """
    Display a hierarchy in various styles.

    Args:
        hierarchy (list): Hierarchical data (list of dicts with 'name' and optionally 'children').
        style (str): The style of display. Options are:
            - "tree" (default): Tree-like with branches.
            - "indented": Simple indented list.
            - "bracketed": Square bracket notation.
            - "table": ASCII table style.
            - "numbered": Numbered outline format.
            - "graph": Arrows to show relationships.
            - "bullets": Bullet points for levels.
            - "arrow": Downward-right arrow for nesting.
            - "json": JSON-like structure.
        level (int): Current recursion level for indentation (internal use).
        prefix (str): Prefix used for numbering or custom markers (internal use).
        start_level (int): Base level for indentation.

    Returns:
        str: The formatted hierarchy as a string.
    """
    output = ""

    for i, item in enumerate(hierarchy, 1):
        # Determine prefixes and indentation
        current_prefix = f"{prefix}{i}" if style == "numbered" else ""
        indent = "    " * (level - start_level)
        marker = ""
        
        if style == "tree":
            marker = f"{'│   ' * (level - start_level)}├── {item['name']}"
        elif style == "indented":
            marker = f"{'  ' * (level - start_level)}- {item['name']}"
        elif style == "bracketed":
            marker = f"{'  ' * (level - start_level)}[{item['name']}]"
        elif style == "table":
            marker = f"{indent}{item['name']}"
        elif style == "numbered":
            marker = f"{current_prefix} {item['name']}"
        elif style == "graph":
            marker = f"{'  ' * (level - start_level)}-> {item['name']}"
        elif style == "bullets":
            marker = f"{'*' * (level - start_level + 1)} {item['name']}"
        elif style == "arrow":
            marker = f"{'    ' * (level - start_level)}↳ {item['name']}"
        elif style == "json":
            marker = f"{indent}{{'name': '{item['name']}'"
            if 'children' in item:
                marker += f", 'children': [\n{display_hierarchy(item['children'], style, level + 1, prefix, start_level)}\n{indent}]"
            marker += f"}}"
        
        # Append the marker to output
        if style != "json":
            output += f"{marker}\n"
        else:
            output += f"{marker}\n"

        # Handle children recursively
        if 'children' in item and style != "json":
            output += display_hierarchy(item['children'], style, level + 1, current_prefix + ".", start_level)

    if show:
    #     print(output)
        print(output)
    return output

In [142]:

import requests

def get_repo_tree(owner, repo, path="src-data", branch = "main"):
    # owner,repo,path,branch = "WCRP-CMIP", "WCRP-UNIVERSE","src-data","main"
    
    if path[-1] == "/": path = path[:-1]
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}/?ref={branch}"


    def tree(url):
        
        owner,repo = url.replace('https://github.com/','').split('/',2)[:2]
        preurl = f"https://{owner}.github.io/{repo}/"
        
        response = requests.get(url)
        contents = response
        if response.status_code == 200:
            contents = response.json()
            
        owner,repo = url.replace('https://api.github.com/repos/','').split('/',2)[:2]
        preurl = f"https://{owner}.github.io/{repo}/"

        for item in contents:
            if '_links' in item: del item['_links']
            item['path']= item['path'].replace(path, '')
            item['jsonld'] = f"{preurl}{item['path']}".split('.json')[0]
            if item['type'] == "dir":
                item['children'] = tree(item['url'])
                
                
        return contents
    
    contents = tree(url)

    complete = {
        "name":repo,
        "path":path,
        "html_url":f"https://github.com/{owner}/{repo}/blob/{branch}/{path}",
        "git_url":url,
        "children":contents
    }

    return complete


contents = get_repo_tree("WCRP-CMIP", "WCRP-UNIVERSE","src-data")

In [ ]:
display_hierarchy([contents],show = True)

In [ ]:
contents['children'][4]

In [143]:
# small = filter_nested_dict(contents)
# small = contents

# for i in small['children']:
#     if 'children' in i:
#         i['size'] = sum([int(j['size']) for j in i['children']])
#     else:
#         i['size'] = int(i['size'])

# small['size'] = sum([int(i['size']) for i in small['children']])  



json.dump(contents,open('test.json','w'))

In [ ]:
contents['size']